# Step 1 - DB ETL and Connection
* Data gathered from a Machine Learning Database: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

In [47]:
# Dependencies
from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy import Index
from sqlalchemy import MetaData
from sqlalchemy import Table
import csv
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, inspect
from sqlalchemy import func

### Data read (csv) and clean up 
* The columns needed to be renamed adding underscores for better manipulation

In [48]:
# Store filepath in a variable
BC_dignosisF = "Resources/data.csv"

# Read our Data file with pandas (Not every CSV requires an encoding, but we added it just in case)
BC_dignosis= pd.read_csv(BC_dignosisF, encoding="utf-8", low_memory=False )
BC_dignosis["diagnosis"] =BC_dignosis["diagnosis"].astype(str)

#Rename the Columns
BC_dignosis.rename(columns={'id': 'patient_id', 'concave points_mean': 'concave_points_mean', 'concave points_se': 'concave_points_se', 'concave points_worst': 'concave_points_worst'}, inplace=True)

#Show
BC_dignosis.head()

,patient_id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [49]:
#Show the data types
BC_dignosis.dtypes.head()

patient_id          int64
diagnosis          object
radius_mean       float64
texture_mean      float64
perimeter_mean    float64
dtype: object

In [50]:
#BC_dignosis.to_csv("Resources/data_renamed.csv",
           #       encoding="utf-8", index=False, header=True)

# Step 2 - Database Set Up 
* Creating db for later manipulation

In [51]:
#Creating db using reflect method
Base = declarative_base()
class BC(Base):
    __tablename__ = 'imagedata_table'
    id = Column(Integer, primary_key=True, nullable=False) 
    patient_id = Column(Integer)  
    diagnosis = Column(String(255))                 
    radius_mean = Column(Float)
    texture_mean = Column(Float)
    perimeter_mean = Column(Float)
    area_mean = Column(Float)
    smoothness_mean= Column(Float)
    compactness_mean = Column(Float)
    concavity_mean = Column(Float)
    concave_points_mean = Column(Float)
    symmetry_mean = Column(Float)
    fractal_dimension_mean= Column(Float)
    radius_se = Column(Float)
    texture_se = Column(Float)
    perimeter_se = Column(Float)
    area_se  = Column(Float)
    smoothness_se = Column(Float)
    compactness_se = Column(Float)
    concavity_se = Column(Float)
    concave_points_se = Column(Float)
    symmetry_se = Column(Float)
    fractal_dimension_se = Column(Float)
    radius_worst = Column(Float)
    texture_worst  = Column(Float)
    perimeter_worst = Column(Float)
    area_worst = Column(Float)
    smoothness_worst = Column(Float)
    compactness_worst = Column(Float)
    concavity_worst = Column(Float)
    concave_points_worst = Column(Float)
    symmetry_worst = Column(Float)
    fractal_dimension_worst = Column(Float)  

#Connecting to db and saving it in the db folder
engine = create_engine("sqlite:///db/BC_diagnosis.db")
Base.metadata.create_all(engine)

#Create the session
session = Session(bind=engine)

In [52]:
#Iterating through rows to input data inside db
for i, row in BC_dignosis.iterrows():
    
    record = BC(**{
        'patient_id': row.patient_id,
        'diagnosis': row.diagnosis, 
        'radius_mean': row.radius_mean, 
        'texture_mean': row.texture_mean, 
        'perimeter_mean': row.perimeter_mean,
        'area_mean': row.area_mean,
        'smoothness_mean': row.smoothness_mean,
        'compactness_mean': row.compactness_mean,
        'concavity_mean': row.concavity_mean,
        'concave_points_mean': row.concave_points_mean, 
        'symmetry_mean': row.symmetry_mean,
        'fractal_dimension_mean': row.fractal_dimension_mean,
        'radius_se': row.radius_se,
        'texture_se': row.texture_se, 
        'perimeter_se': row.perimeter_se, 
        'area_se': row.area_se, 
        'smoothness_se': row.smoothness_se,
        'compactness_se': row.compactness_se,
        'concavity_se': row.concavity_se,
        'concave_points_se': row.concave_points_se, 
        'symmetry_se': row.symmetry_se,
        'fractal_dimension_se': row.fractal_dimension_se, 
        'radius_worst': row.radius_worst, 
        'texture_worst': row.texture_worst,
        'perimeter_worst': row.perimeter_worst,
        'area_worst': row.area_worst, 
        'smoothness_worst': row.smoothness_worst,
        'compactness_worst': row.compactness_worst,
        'concavity_worst': row.concavity_worst, 
        'concave_points_worst': row.concave_points_worst,
        'symmetry_worst': row.symmetry_worst, 
        'fractal_dimension_worst': row.fractal_dimension_worst
      
})

    session.add(record)
session.commit()

In [53]:
#Display row
row.head()

patient_id        92751
diagnosis             B
radius_mean        7.76
texture_mean      24.54
perimeter_mean    47.92
Name: 568, dtype: object

In [54]:
Base = declarative_base()
Base

sqlalchemy.ext.declarative.api.Base

In [55]:
engine = create_engine("sqlite:///db/BC_diagnosis.db")
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['imagedata_table']

In [56]:
#Check that db is working by inspecting
#Use reflect method 
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.engine.reflection import Inspector

meta = MetaData()
user_table = Table('imagedata_table', meta)
insp = Inspector.from_engine(engine)
insp.reflecttable(user_table, None)

In [57]:
disgnosis=Base.classes.imagedata_table
disgnosis

sqlalchemy.ext.automap.imagedata_table

In [58]:
inspector = inspect(engine)
inspector.get_table_names()

['imagedata_table']

In [59]:
columns = inspector.get_columns('imagedata_table')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
patient_id INTEGER
diagnosis VARCHAR(255)
radius_mean FLOAT
texture_mean FLOAT
perimeter_mean FLOAT
area_mean FLOAT
smoothness_mean FLOAT
compactness_mean FLOAT
concavity_mean FLOAT
concave_points_mean FLOAT
symmetry_mean FLOAT
fractal_dimension_mean FLOAT
radius_se FLOAT
texture_se FLOAT
perimeter_se FLOAT
area_se FLOAT
smoothness_se FLOAT
compactness_se FLOAT
concavity_se FLOAT
concave_points_se FLOAT
symmetry_se FLOAT
fractal_dimension_se FLOAT
radius_worst FLOAT
texture_worst FLOAT
perimeter_worst FLOAT
area_worst FLOAT
smoothness_worst FLOAT
compactness_worst FLOAT
concavity_worst FLOAT
concave_points_worst FLOAT
symmetry_worst FLOAT
fractal_dimension_worst FLOAT


In [60]:
from sqlalchemy import inspect

mapper = inspect(BC)
print(mapper)

mapped class BC->imagedata_table


In [61]:
Session = sessionmaker(bind=engine)
# Session is a class
session = Session()
# now session is a instance of the class Session

In [62]:
engine.execute('SELECT * FROM imagedata_table LIMIT 5').fetchall()

[(1, 842302, 'M', 17.99, 10.38, 122.8, 1001.0, 0.1184, 0.2776, 0.3001, 0.1471, 0.2419, 0.07871, 1.095, 0.9053, 8.589, 153.4, 0.006399, 0.04904, 0.05372999999999999, 0.01587, 0.03003, 0.006193, 25.38, 17.33, 184.6, 2019.0, 0.1622, 0.6656, 0.7119, 0.2654, 0.4601, 0.1189),
 (2, 842517, 'M', 20.57, 17.77, 132.9, 1326.0, 0.08474, 0.07864, 0.0869, 0.07017000000000001, 0.1812, 0.056670000000000005, 0.5435, 0.7339, 3.398, 74.08, 0.005225, 0.013080000000000001, 0.0186, 0.0134, 0.013890000000000001, 0.003532, 24.99, 23.41, 158.8, 1956.0, 0.1238, 0.1866, 0.2416, 0.18600000000000003, 0.275, 0.08902),
 (3, 84300903, 'M', 19.69, 21.25, 130.0, 1203.0, 0.1096, 0.1599, 0.1974, 0.1279, 0.2069, 0.059989999999999995, 0.7456, 0.7869, 4.585, 94.03, 0.00615, 0.040060000000000005, 0.03832, 0.02058, 0.0225, 0.004571, 23.57, 25.53, 152.5, 1709.0, 0.1444, 0.4245, 0.4504, 0.243, 0.3613, 0.08757999999999999),
 (4, 84348301, 'M', 11.42, 20.38, 77.58, 386.1, 0.1425, 0.2839, 0.2414, 0.1052, 0.2597, 0.09744, 0.4956, 1